# 🖼️ Offline Image Caption Generator
### Vision-Language Transformer (BLIP) — Google Colab

**What this does:**  
Generates natural language captions for any image using a pretrained BLIP (Bootstrapped Language-Image Pretraining) model from Salesforce. No API key needed — model weights download once from HuggingFace and run locally on Colab.

---
**Steps:**
1. Install dependencies
2. Load BLIP model
3. Upload your own image OR use a URL
4. Generate & display captions
5. Batch caption multiple images

> ⚡ **Tip:** Go to `Runtime → Change runtime type → T4 GPU` for faster inference

## 📦 Step 1 — Install Dependencies

In [ ]:
# Install required libraries
!pip install transformers torch torchvision Pillow -q
print("✅ All dependencies installed!")

## 📚 Step 2 — Import Libraries

In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from google.colab import files
import os
import warnings
warnings.filterwarnings('ignore')

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Running on: {device.upper()}")
if device == "cuda":
    print(f"⚡ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("💡 Tip: Enable GPU from Runtime → Change runtime type → T4 GPU")

## 🤖 Step 3 — Load BLIP Model

> Downloads ~1GB weights on first run. Cached after that — subsequent runs are instant.

In [ ]:
MODEL_NAME = "Salesforce/blip-image-captioning-large"

print(f"📥 Loading model: {MODEL_NAME}")
print("   (First run downloads ~1GB — grab a coffee ☕)")

processor = BlipProcessor.from_pretrained(MODEL_NAME)
model     = BlipForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

model.eval()
print("\n✅ Model loaded and ready!")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

## 🛠️ Step 4 — Caption Generator Function

In [ ]:
def generate_caption(image: Image.Image,
                     prompt: str = None,
                     max_length: int = 50,
                     num_captions: int = 3,
                     num_beams: int = 5) -> list:
    """
    Generate captions for a PIL image.

    Args:
        image        : PIL.Image — input image
        prompt       : optional text prompt to guide captioning
        max_length   : max caption length in tokens
        num_captions : number of diverse captions to return
        num_beams    : beam search width (higher = better quality, slower)

    Returns:
        list of caption strings
    """
    image = image.convert("RGB")

    # Prepare inputs
    if prompt:
        inputs = processor(image, prompt, return_tensors="pt").to(device)
    else:
        inputs = processor(image, return_tensors="pt").to(device)

    # Cast to float16 if on GPU
    if device == "cuda":
        inputs = {k: v.half() if v.dtype == torch.float32 else v
                  for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=num_captions,
            early_stopping=True,
            repetition_penalty=1.3,
        )

    captions = processor.batch_decode(output_ids, skip_special_tokens=True)
    return [c.strip().capitalize() for c in captions]


def show_image_with_captions(image: Image.Image, captions: list, title: str = ""):
    """Display image alongside generated captions."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5),
                              gridspec_kw={'width_ratios': [1, 1]})

    # Left: image
    axes[0].imshow(image)
    axes[0].axis('off')
    axes[0].set_title(title or "Input Image", fontsize=13, fontweight='bold', pad=10)

    # Right: captions
    axes[1].axis('off')
    axes[1].set_facecolor('#f8f9fa')
    caption_text = "🤖 Generated Captions\n" + "─" * 38 + "\n\n"
    for i, cap in enumerate(captions, 1):
        caption_text += f"{i}. {cap}\n\n"
    axes[1].text(0.05, 0.95, caption_text,
                 transform=axes[1].transAxes,
                 fontsize=11, verticalalignment='top',
                 fontfamily='monospace',
                 bbox=dict(boxstyle='round', facecolor='#eef2ff', alpha=0.8))

    plt.tight_layout()
    plt.show()

print("✅ Helper functions ready!")

## 🖼️ Step 5A — Upload YOUR Image

Run this cell to upload a JPG / PNG image from your computer.

In [ ]:
# Upload your own image
print("📂 Select an image to upload...")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    image    = Image.open(filename).convert("RGB")

    print(f"\n✅ Uploaded: {filename}")
    print(f"   Size: {image.size[0]}x{image.size[1]} px")
    print("\n⏳ Generating captions...")

    captions = generate_caption(image, num_captions=3, num_beams=5)
    show_image_with_captions(image, captions, title=filename)

    print("\n📝 Captions:")
    for i, cap in enumerate(captions, 1):
        print(f"   {i}. {cap}")
else:
    print("⚠️  No file uploaded. Try again or use Step 5B below.")

## 🌐 Step 5B — Use an Image URL (No Upload Needed)

In [ ]:
# ─── CHANGE THIS URL to any image you want ───────────────────────────
IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Cute_dog.jpg/1200px-Cute_dog.jpg"
# ─────────────────────────────────────────────────────────────────────

print(f"🌐 Fetching image from URL...")
response = requests.get(IMAGE_URL, timeout=10)
image    = Image.open(BytesIO(response.content)).convert("RGB")

print(f"✅ Image loaded — Size: {image.size[0]}x{image.size[1]} px")
print("⏳ Generating captions...")

captions = generate_caption(image, num_captions=3, num_beams=5)
show_image_with_captions(image, captions, title="Image from URL")

print("\n📝 Captions:")
for i, cap in enumerate(captions, 1):
    print(f"   {i}. {cap}")

## 💬 Step 6 — Prompted Captioning (Guide the Output)

Add a text prompt to steer the caption toward a specific aspect of the image.

In [ ]:
# ─── CHANGE these to your image + prompt ─────────────────────────────
IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Cute_dog.jpg/1200px-Cute_dog.jpg"
PROMPT    = "a photo of"   # try: "the background shows", "the emotion is", "there is"
# ─────────────────────────────────────────────────────────────────────

response = requests.get(IMAGE_URL, timeout=10)
image    = Image.open(BytesIO(response.content)).convert("RGB")

print(f"💬 Prompt: '{PROMPT}'")
print("⏳ Generating prompted captions...")

captions = generate_caption(image, prompt=PROMPT, num_captions=3, num_beams=5)
show_image_with_captions(image, captions, title=f"Prompted: '{PROMPT}'")

print("\n📝 Prompted Captions:")
for i, cap in enumerate(captions, 1):
    print(f"   {i}. {cap}")

## 📁 Step 7 — Batch Caption Multiple Images

Upload a folder or list of images — captions are generated for all and saved to a CSV.

In [ ]:
import pandas as pd
import time

# ─── List of image URLs to batch process ─────────────────────────────
IMAGE_URLS = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Cute_dog.jpg/1200px-Cute_dog.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/280px-PNG_transparency_demonstration_1.png",
]
# ─────────────────────────────────────────────────────────────────────

results = []
print(f"⏳ Processing {len(IMAGE_URLS)} images...\n")

for idx, url in enumerate(IMAGE_URLS, 1):
    try:
        t0       = time.time()
        response = requests.get(url, timeout=10)
        image    = Image.open(BytesIO(response.content)).convert("RGB")
        captions = generate_caption(image, num_captions=1, num_beams=4)
        elapsed  = time.time() - t0

        results.append({"image": url.split("/")[-1], "caption": captions[0], "time_s": round(elapsed, 2)})
        print(f"  [{idx}/{len(IMAGE_URLS)}] ✅ {url.split('/')[-1]}")
        print(f"        → {captions[0]}")
        print(f"        ⏱  {elapsed:.2f}s\n")

    except Exception as e:
        print(f"  [{idx}] ❌ Failed: {e}")
        results.append({"image": url, "caption": "ERROR", "time_s": 0})

# Save results
df = pd.DataFrame(results)
df.to_csv("captions_output.csv", index=False)
print("\n📊 Results:")
print(df.to_string(index=False))
print("\n💾 Saved to captions_output.csv")

## 📊 Step 8 — Evaluate with BLEU Score

Compare generated captions against reference captions using BLEU metric.

In [ ]:
!pip install nltk -q
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
nltk.download('punkt', quiet=True)

# ─── Add your reference captions here ────────────────────────────────
reference_captions = [
    "a dog sitting on a grassy field",
    "a cute brown dog is looking at the camera",
    "a dog outdoors in nature"
]
# ─────────────────────────────────────────────────────────────────────

IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Cute_dog.jpg/1200px-Cute_dog.jpg"
response  = requests.get(IMAGE_URL, timeout=10)
image     = Image.open(BytesIO(response.content)).convert("RGB")

generated = generate_caption(image, num_captions=1, num_beams=5)[0].lower()

# Tokenize
refs      = [ref.split() for ref in reference_captions]
hyp       = generated.split()
smoother  = SmoothingFunction().method1

bleu1 = sentence_bleu(refs, hyp, weights=(1,0,0,0), smoothing_function=smoother)
bleu2 = sentence_bleu(refs, hyp, weights=(0.5,0.5,0,0), smoothing_function=smoother)
bleu4 = sentence_bleu(refs, hyp, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoother)

print("📊 BLEU Score Evaluation")
print("=" * 40)
print(f"Generated : {generated}")
print(f"References: {reference_captions}")
print()
print(f"  BLEU-1 : {bleu1:.4f}")
print(f"  BLEU-2 : {bleu2:.4f}")
print(f"  BLEU-4 : {bleu4:.4f}")

---
## ✅ Summary

| Feature | Details |
|---|---|
| **Model** | `Salesforce/blip-image-captioning-large` |
| **Architecture** | Vision Transformer (ViT) + BERT decoder |
| **Input** | JPG / PNG — upload, URL, or batch |
| **Output** | 1–3 natural language captions per image |
| **Evaluation** | BLEU-1, BLEU-2, BLEU-4 |
| **Extras** | Prompted captioning, batch CSV export |

---
**Author:** Maridi Lakshminarayana · [github.com/Lakshminarayan566](https://github.com/Lakshminarayan566) · maridi.lakshminarayana111@gmail.com